# Aufgabenstellungen Modul 2

## Aufgabenstellung Teilmodul 2.1

### Auswahl und Zusammenführen Ihrer Daten
Suchen Sie auf [bibsonomy](https://www.bibsonomy.org) mit einem selbst gewählten Suchbegriff nach Publikationen. Auf [bibsonomy](https://www.bibsonomy.org) lassen sich die Ergebnisse einer Keyword-Suche (z.B. ["Bibliothek"](https://www.bibsonomy.org/export/search/Bibliothek)) in einer Vielzahl verschiedener Formate exportieren (Limit ggf. vorher auf 1000 erhöhen) und lokal speichern. Geben Sie dafür einen Suchbegriff oben rechts in die Maske ein und bestätigen Sie. Rechts neben der Überschrift der rechten Spalte ("publications"/"Publikationen") klicken Sie ganz rechte auf das Symbol mit dem Pfeil nach rechts, der aus einem Kasten zeigt ("export options for displayed posts"/"Exportieren für angezeigte Einträge"), dann auf mehr/more. Wählen Sie ZUERST bei posts/Einträge "1000" aus, und wählen sie dann in dem Format drop-Down-Menü fast ganz unten JSON aus. Es öffnet sich die JSON Datei in ihrem Browser.  Sie können die URL wie im Tutorial über das requests Modul in Python öffnen.
Alternativ können Sie die Datei auch lokal speichern. Dafür rechtsklicken Sie irgendwo, wählen Sie "save as..."/"speichern unter..." und speichern Sie die Datei unter einem passenden Dateinamen mit der Endung ".json".

Für eine Nutzung im zweiten Modulteil ist ein großer Datensatz sinnvoll.Fügen sie daher die Ergebnisse mehrerer Suchen mit Pandas in einem Datensatz zusammenführen. 
Fügen sie ihren Datensätzen dabei den Suchterm hinzu, der zu den Ergebnissen geführt hat. Tipp: Informationen dazu finden Sie unter "Daten Verändern und Hinzufügen".

Nutzen sie dabei Python (vorzugsweise in einem Jupyter Notebook) mithilfe von Pandas. Sollten Sie Probleme haben, schauen Sie im Pandas Tutorial unter "JSON einlesen" oder bitten Sie Fabian Haak im Moodle Austauschforum um Hilfe.

Legen Sie Ihr Notebook bzw. Python-Script mit den Lösungen in einem "Modul_2-1" Ordner im eigenen Repositorium auf GitHub ab. Fügen Sie mich als zusätzlichen Collaborator (GitHub-Name: HaakFab) hinzu, so kann ich besser Feedback oder Hilfestellungen geben.

### Exportformat erstellen

Diese Teilaufgabe dient als direkte Vorbereitung des nächsten Modulteils. Wir empfehlen, Pandas für die Bearbeitung der Aufgabe zu nutzen. 

#### Datenaufbereitung
Für den nächsten Teil dieses Modules möchten wir unseren Datensatz wieder als JSON-File ablegen. Zunächst möchten wir aber die Bibsonomy-Daten  noch etwas zu bereinigen:
- Filtern Sie die Daten so, dass nur noch Einträge des Typs "Publication" enthalten sind.
- Überlegen Sie, welche Felder Sie möglicherweise umbenennen wollen, und welche Sie nicht benötigen. Benennen Sie entsprechend um und wählen Sie nur die Spalten, die für Sie sinnvolle Informationen enthalten. Mögliche Änderungen der Daten sind das Umbenennen von Felder zu eindeutigeren Bezeichnungen (wie beispielsweise im Pandas Tutorial "label" zu "title"). Filtern Sie ihre Daten jedoch nicht zu drastisch und entfernen Sie nicht zu viele Felder, damit im zweiten Modulteil noch genügend Informationen übrig bleiben. 
- Wenn sie mehrere Suchergebnisse zusammengeführt haben, ist es sinnvoll, Nachduplikaten zu überprüfen.
- BONUS: Wie bei den Jareszahlen ist es durchaus sinnvoll, zu überprüfen, ob alle Einträge "sauber" vorliegen. Stellen sie dafür sicher, dass alle Einträge Informationen zu Titel, AutorInnen und Veröffentlichungsjahr enthalten und dass das Jahr eine sinnvolle Zahl ist!

#### Daten Export
Erstellen Sie einen sauberen Export im JSON-Format, das von Solr akzeptiert wird.

Hier ein Beispiel, das zeigt, wie die Struktur aussehen soll:

```json
[
  {
    "id" : "978-0641723445",
    "cat" : ["book","hardcover"],
    "name" : "The Lightning Thief",
    "author" : "Rick Riordan",
    "series_t" : "Percy Jackson and the Olympians",
    "sequence_i" : 1,
    "genre_s" : "fantasy",
    "inStock" : true,
    "price" : 12.50,
    "pages_i" : 384
  }
,
  {
    "id" : "978-1423103349",
    "cat" : ["book","paperback"],
    "name" : "The Sea of Monsters",
    "author" : "Rick Riordan",
    "series_t" : "Percy Jackson and the Olympians",
    "sequence_i" : 2,
    "genre_s" : "fantasy",
    "inStock" : true,
    "price" : 6.49,
    "pages_i" : 304
  }
]
```

Hinweis: Für die Weitervervendung ist nicht relevant, wie die einzelnen Felder benannt sind oder wie viele Felder es gibt, Es ist aber entscheidend, dass die äußere Form einer Liste von Dictionaries gewahrt wird. (HINWEIS BEI PROBLEMEN: Schauen Sie im Pandas Tutorial unter "Export als JSON")

In [2]:
import pandas as pd
import json
import requests
import urllib.parse

from pathlib import Path

In [10]:
def df_from_query(query, limit=1000):
    """
    Create a DataFrame by querying BibSonomy for specific search terms.

    Args:
        query (str or list of str): The search query or a list of search queries to retrieve data for.
        limit (int, optional): The maximum number of items to retrieve for each query. Defaults to 1000.

    Returns:
        pandas.DataFrame: A DataFrame containing the retrieved data from BibSonomy, with an additional 'search_string' column
        that indicates the corresponding search query for each row.
    """
    # Check datatype of query
    if not isinstance(query, list):
        if isinstance(query, str):
            query = [query]
        else:
            print('Please provide a valid query (string or list of strings).')
    # Query bibsonomy
    dfs = []
    dfs_dict = {}
    for q in query:
        url = f"https://www.bibsonomy.org/json/search/{urllib.parse.quote(q)}?duplicates=merged&items={limit}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data['items'])
            df['search_string'] = q
            print(f"Query: '{q}'. Number of items retrieved: {len(df)}")
            dfs.append(df) # to list
            dfs_dict[q] = df # to dict
        else:
            print(f'{response.status_code}: {url} is not available.')
    # Concat DataFrames
    df = pd.concat(dfs, ignore_index=True)
    return df, dfs_dict

In [11]:
queries = ['makerspace', 'virtual reality', 'mixed reality', 'augmented reality']

df, dfs_dict = df_from_query(queries, limit=1000)

Query: 'makerspace'. Number of items retrieved: 187
Query: 'virtual reality'. Number of items retrieved: 1335
Query: 'mixed reality'. Number of items retrieved: 1071
Query: 'augmented reality'. Number of items retrieved: 1256


In [14]:
dfs_dict.keys()

dict_keys(['makerspace', 'virtual reality', 'mixed reality', 'augmented reality'])

In [46]:
df.head()

,type,id,tags,intraHash,label,user,description,date,changeDate,count,...,citeulike-linkout-3,opac,status,oai-id,oai-set,unit,details,documenturl,review,journaltitle
0,Bookmark,https://www.bibsonomy.org/url/a10ba61e8b7f7ca6...,[Unsortierte_Lesezeichen],a10ba61e8b7f7ca6db55da2670090ef2,Attraktor - Der Makerspace in Hamburg,robingarcia,Der gemeinnützige Attraktor e. V. betreibt den...,2017-03-17 10:34:46,2017-04-13 09:59:00,1,...,None,None,None,None,None,None,None,None,None,None
1,Bookmark,https://www.bibsonomy.org/url/6093e51d381e6c5c...,"[educação, maker]",6093e51d381e6c5c2a7d5ee954f7c1ee,Why Makerspaces are Changing the World – Betab...,christianoavila,While in college I learned about engineering i...,2017-05-09 12:48:55,2017-05-09 12:48:55,1,...,None,None,None,None,None,None,None,None,None,None
2,Bookmark,https://www.bibsonomy.org/url/90a1ee85e2acf284...,"[thinking, design, makerspace, makers, learnin...",90a1ee85e2acf284765373ea8756d255,Design Thinking Process and UDL Planning Tool ...,yish,"If there is a makerspace in your school, it ma...",2017-12-25 16:06:35,2017-12-25 16:06:35,1,...,None,None,None,None,None,None,None,None,None,None
3,Bookmark,https://www.bibsonomy.org/url/f1009e9d37cb7bfd...,"[labs, tesis, maker]",f1009e9d37cb7bfdf0208ac6d7f052b4,Albemarle County Schools’ Journey From a Maker...,djimenezsanchez,,2016-08-04 09:40:49,2016-08-04 09:40:49,1,...,None,None,None,None,None,None,None,None,None,None
4,Bookmark,https://www.bibsonomy.org/url/5d1439dd75497437...,"[makerspace, makers, israel, education, maker]",5d1439dd7549743700296c739da928ee,XLN - Anything is possible. Anyone Can.,yish,,2017-06-08 14:45:40,2017-06-08 14:45:40,1,...,None,None,None,None,None,None,None,None,None,None


In [47]:
df.shape

(3849, 128)

In [48]:
df['search_string'].value_counts()

search_string
virtual reality      1335
augmented reality    1256
mixed reality        1071
makerspace            187
Name: count, dtype: int64

In [49]:
df.columns

Index(['type', 'id', 'tags', 'intraHash', 'label', 'user', 'description',
       'date', 'changeDate', 'count',
       ...
       'citeulike-linkout-3', 'opac', 'status', 'oai-id', 'oai-set', 'unit',
       'details', 'documenturl', 'review', 'journaltitle'],
      dtype='object', length=128)

In [50]:
df['label'].nunique()

3513

In [51]:
# Save to json
json_file = Path('data/makerspace-bibsonomy-raw.json')

In [52]:
df.to_json(json_file, orient="records")

# Data Cleaning

In [53]:
# Load json
df = pd.read_json(json_file)

In [54]:
df.head(3)

,type,id,tags,intraHash,label,user,description,date,changeDate,count,...,citeulike-linkout-3,opac,status,oai-id,oai-set,unit,details,documenturl,review,journaltitle
0,Bookmark,https://www.bibsonomy.org/url/a10ba61e8b7f7ca6...,[Unsortierte_Lesezeichen],a10ba61e8b7f7ca6db55da2670090ef2,Attraktor - Der Makerspace in Hamburg,robingarcia,Der gemeinnützige Attraktor e. V. betreibt den...,2017-03-17 10:34:46,2017-04-13 09:59:00,1,...,None,None,None,None,None,None,None,None,None,None
1,Bookmark,https://www.bibsonomy.org/url/6093e51d381e6c5c...,"[educação, maker]",6093e51d381e6c5c2a7d5ee954f7c1ee,Why Makerspaces are Changing the World – Betab...,christianoavila,While in college I learned about engineering i...,2017-05-09 12:48:55,2017-05-09 12:48:55,1,...,None,None,None,None,None,None,None,None,None,None
2,Bookmark,https://www.bibsonomy.org/url/90a1ee85e2acf284...,"[thinking, design, makerspace, makers, learnin...",90a1ee85e2acf284765373ea8756d255,Design Thinking Process and UDL Planning Tool ...,yish,"If there is a makerspace in your school, it ma...",2017-12-25 16:06:35,2017-12-25 16:06:35,1,...,None,None,None,None,None,None,None,None,None,None


In [55]:
df['abstract'].nunique()

416

In [56]:
# Get only columns with more than 400 non null values (400 = to get all entries with abstracts)
# All columns below 400 non null values are dropped
non_null_count = df.count() 
usable_columns = non_null_count[non_null_count >= 400]

df_clean = df.filter(usable_columns.index)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3849 entries, 0 to 3848
Data columns (total 30 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   type           3849 non-null   object        
 1   id             3849 non-null   object        
 2   tags           3849 non-null   object        
 3   intraHash      3849 non-null   object        
 4   label          3849 non-null   object        
 5   user           3849 non-null   object        
 6   description    3849 non-null   object        
 7   date           3849 non-null   datetime64[ns]
 8   changeDate     3849 non-null   object        
 9   count          3849 non-null   int64         
 10  url            3849 non-null   object        
 11  interHash      3149 non-null   object        
 12  pub-type       3149 non-null   object        
 13  booktitle      2090 non-null   object        
 14  publisher      2032 non-null   object        
 15  year           3149 n

In [57]:
df_clean.head()

,type,id,tags,intraHash,label,user,description,date,changeDate,count,...,pages,abstract,isbn,bibtexKey,journal,series,volume,number,ee,search_string
0,Bookmark,https://www.bibsonomy.org/url/a10ba61e8b7f7ca6...,[Unsortierte_Lesezeichen],a10ba61e8b7f7ca6db55da2670090ef2,Attraktor - Der Makerspace in Hamburg,robingarcia,Der gemeinnützige Attraktor e. V. betreibt den...,2017-03-17 10:34:46,2017-04-13 09:59:00,1,...,None,None,None,None,None,None,None,None,None,makerspace
1,Bookmark,https://www.bibsonomy.org/url/6093e51d381e6c5c...,"[educação, maker]",6093e51d381e6c5c2a7d5ee954f7c1ee,Why Makerspaces are Changing the World – Betab...,christianoavila,While in college I learned about engineering i...,2017-05-09 12:48:55,2017-05-09 12:48:55,1,...,None,None,None,None,None,None,None,None,None,makerspace
2,Bookmark,https://www.bibsonomy.org/url/90a1ee85e2acf284...,"[thinking, design, makerspace, makers, learnin...",90a1ee85e2acf284765373ea8756d255,Design Thinking Process and UDL Planning Tool ...,yish,"If there is a makerspace in your school, it ma...",2017-12-25 16:06:35,2017-12-25 16:06:35,1,...,None,None,None,None,None,None,None,None,None,makerspace
3,Bookmark,https://www.bibsonomy.org/url/f1009e9d37cb7bfd...,"[labs, tesis, maker]",f1009e9d37cb7bfdf0208ac6d7f052b4,Albemarle County Schools’ Journey From a Maker...,djimenezsanchez,,2016-08-04 09:40:49,2016-08-04 09:40:49,1,...,None,None,None,None,None,None,None,None,None,makerspace
4,Bookmark,https://www.bibsonomy.org/url/5d1439dd75497437...,"[makerspace, makers, israel, education, maker]",5d1439dd7549743700296c739da928ee,XLN - Anything is possible. Anyone Can.,yish,,2017-06-08 14:45:40,2017-06-08 14:45:40,1,...,None,None,None,None,None,None,None,None,None,makerspace


In [58]:
# Only work with entires of type 'Publication'
df_clean = df_clean.query('type == "Publication"')
df_clean.head()

,type,id,tags,intraHash,label,user,description,date,changeDate,count,...,pages,abstract,isbn,bibtexKey,journal,series,volume,number,ee,search_string
38,Publication,https://www.bibsonomy.org/bibtex/2e033978aa497...,[greenbib],e033978aa4971fb861cc33beb8c3bb7c,How to start a library makerspace,bibgreen,,2023-02-05 21:50:14,2023-02-16 11:47:43,1,...,3-18,You may have heard the term ``makerspace'' and...,978-0-8389-1504-2,Bronkar.2017,None,None,None,None,None,makerspace
39,Publication,https://www.bibsonomy.org/bibtex/2fb48ac294725...,"[assessment, outcomes, makerspace, learningana...",fb48ac2947255e4aca45f9dcc913a382,Informing Makerspace Outcomes Through a Lingui...,ereidt,,2020-02-18 21:35:21,2020-02-18 21:35:21,1,...,None,A growing body of research focuses on what out...,None,Oliver2020,International Journal of Science and Mathemati...,None,None,None,None,makerspace
40,Publication,https://www.bibsonomy.org/bibtex/2789d019603b4...,[greenbib],789d019603b437609dec3272a28f24a9,The makerspace librarian's sourcebook,bibgreen,,2023-02-05 21:52:55,2023-02-05 21:52:55,1,...,None,The Makerspace Librarian's Sourcebook is an es...,978-0-8389-1504-2,Kroski.2017,None,None,None,None,None,makerspace
41,Publication,https://www.bibsonomy.org/bibtex/2309634b7a919...,[greenbib],309634b7a919e39e8f8f3289b783ff57,Terrific makerspace projects: A practical guid...,bibgreen,,2023-02-05 16:28:34,2023-02-05 16:28:34,1,...,None,Step-by-step instructions to guide you through...,9781538131824,Denzer.2020,None,Practical guides for librarians,no. 67,None,None,makerspace
42,Publication,https://www.bibsonomy.org/bibtex/21c020a4a195d...,[greenbib],1c020a4a195d8f9dcad4ed295bfbf0e5,Sustainability: Keeping the library makerspace...,bibgreen,,2023-02-16 11:58:46,2023-02-16 11:58:46,1,...,325-344,None,978-0-8389-1504-2,Ginsberg.2017,None,None,None,None,None,makerspace


In [59]:
df_clean.columns

Index(['type', 'id', 'tags', 'intraHash', 'label', 'user', 'description',
       'date', 'changeDate', 'count', 'url', 'interHash', 'pub-type',
       'booktitle', 'publisher', 'year', 'author', 'authors', 'editor',
       'editors', 'pages', 'abstract', 'isbn', 'bibtexKey', 'journal',
       'series', 'volume', 'number', 'ee', 'search_string'],
      dtype='object')

## Clean column `year`

In [60]:
df_clean[~df_clean['year'].str.match(r'\d{4}')][['year', 'label']]

,year,label
1019,2021,Fostering Teacher Educator Technology Competen...
3100,2021,Evaluation Design Methodology for Piloting Two...
3103,2021,Evaluation Design Methodology for Piloting Two...


In [61]:
df_clean['year'] = df_clean['year'].str.strip().replace(r'\D+', '', regex=True)
df_clean[~df_clean['year'].str.match(r'\d{4}')][['year', 'label']]

,year,label


In [62]:
df_clean['year'] = pd.to_datetime(df_clean['year']).dt.year
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3149 entries, 38 to 3848
Data columns (total 30 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   type           3149 non-null   object        
 1   id             3149 non-null   object        
 2   tags           3149 non-null   object        
 3   intraHash      3149 non-null   object        
 4   label          3149 non-null   object        
 5   user           3149 non-null   object        
 6   description    3149 non-null   object        
 7   date           3149 non-null   datetime64[ns]
 8   changeDate     3149 non-null   object        
 9   count          3149 non-null   int64         
 10  url            3149 non-null   object        
 11  interHash      3149 non-null   object        
 12  pub-type       3149 non-null   object        
 13  booktitle      2090 non-null   object        
 14  publisher      2032 non-null   object        
 15  year           3149 non-n

## Clean `authors` column

In [63]:
len(df_clean[df_clean['author'].isna()])

37

In [64]:
df_clean[df_clean['author'].notna()][['author', 'authors']]

,author,authors
38,[Cherie Bronkar],"[{'first': 'Cherie', 'last': 'Bronkar'}]"
39,"[Kevin M. Oliver, Jennifer K. Houchins, Robert...","[{'first': 'Kevin M.', 'last': 'Oliver'}, {'fi..."
41,"[Juan Denzer, Sharona Ginsberg]","[{'first': 'Juan', 'last': 'Denzer'}, {'first'..."
42,[Sharona Ginsberg],"[{'first': 'Sharona', 'last': 'Ginsberg'}]"
43,"[John J. Burke, Ellyssa Kroski]","[{'first': 'John J.', 'last': 'Burke'}, {'firs..."
...,...,...
3844,"[Ángel del Río, Jan Fischer, M. Köbele, Dirk B...","[{'first': 'Ángel', 'last': 'del Río'}, {'firs..."
3845,"[Leif Oppermann, Lisa Blum, Junyeong Lee, Jung...","[{'first': 'Leif', 'last': 'Oppermann'}, {'fir..."
3846,"[Miguel Ribo, Peter Lang, Harald Ganster, Mark...","[{'first': 'Miguel', 'last': 'Ribo'}, {'first'..."
3847,"[I. Poupyrev, R. Berry, M. Billinghurst, H. Ka...","[{'first': 'I.', 'last': 'Poupyrev'}, {'first'..."


In [65]:
# Split column 'author' to get first_author for sorting purposes
def get_first_author(names):
    if isinstance(names, list):
        first_author = names[0].split()
        first_author = [first_author[-1], ' '.join(first_author[0:-1])]
        first_author = ', '.join(first_author)
        return first_author
    else:
        return None

df_clean['author_first'] = df_clean['author'].apply(get_first_author)

In [66]:
# Save author count
df_clean['author_count'] = df_clean['author'].apply(lambda author: len(author) if isinstance(author, list) else 0)

### `drop_na` authors

In [68]:
df_clean = df_clean.dropna(subset='author')

In [69]:
df_clean.head()

,type,id,tags,intraHash,label,user,description,date,changeDate,count,...,isbn,bibtexKey,journal,series,volume,number,ee,search_string,author_first,author_count
38,Publication,https://www.bibsonomy.org/bibtex/2e033978aa497...,[greenbib],e033978aa4971fb861cc33beb8c3bb7c,How to start a library makerspace,bibgreen,,2023-02-05 21:50:14,2023-02-16 11:47:43,1,...,978-0-8389-1504-2,Bronkar.2017,None,None,None,None,None,makerspace,"Bronkar, Cherie",1
39,Publication,https://www.bibsonomy.org/bibtex/2fb48ac294725...,"[assessment, outcomes, makerspace, learningana...",fb48ac2947255e4aca45f9dcc913a382,Informing Makerspace Outcomes Through a Lingui...,ereidt,,2020-02-18 21:35:21,2020-02-18 21:35:21,1,...,None,Oliver2020,International Journal of Science and Mathemati...,None,None,None,None,makerspace,"Oliver, Kevin M.",4
41,Publication,https://www.bibsonomy.org/bibtex/2309634b7a919...,[greenbib],309634b7a919e39e8f8f3289b783ff57,Terrific makerspace projects: A practical guid...,bibgreen,,2023-02-05 16:28:34,2023-02-05 16:28:34,1,...,9781538131824,Denzer.2020,None,Practical guides for librarians,no. 67,None,None,makerspace,"Denzer, Juan",2
42,Publication,https://www.bibsonomy.org/bibtex/21c020a4a195d...,[greenbib],1c020a4a195d8f9dcad4ed295bfbf0e5,Sustainability: Keeping the library makerspace...,bibgreen,,2023-02-16 11:58:46,2023-02-16 11:58:46,1,...,978-0-8389-1504-2,Ginsberg.2017,None,None,None,None,None,makerspace,"Ginsberg, Sharona",1
43,Publication,https://www.bibsonomy.org/bibtex/2b80b466b358b...,[greenbib],b80b466b358b345222205aee793721df,Makerspaces: A practical guide for librarians,bibgreen,,2023-02-05 16:08:03,2023-02-05 16:08:03,1,...,978-1538108185,Burke.2018,None,Practical guides for librarians,no. 38,None,None,makerspace,"Burke, John J.",2


In [37]:
df_clean = df_clean.filter(['search_string', 'type', 'pub-type', 'year', 'author_first', 'authors', 'author',
                            'author_count', 'label', 'abstract', 'booktitle', 'journal', 'tags', 'isbn', 
                            'count', 'id', 'bibtexKey', 'date', 'changeDate'])
df_clean = df_clean.rename(columns={'label': 'title'})
df_clean.head()

,search_string,type,pub-type,year,author_first,authors,author,author_count,title,abstract,booktitle,journal,tags,isbn,count,id,bibtexKey,date,changeDate
38,makerspace,Publication,incollection,2017,"Bronkar, Cherie","[{'first': 'Cherie', 'last': 'Bronkar'}]",[Cherie Bronkar],1,How to start a library makerspace,You may have heard the term ``makerspace'' and...,The makerspace librarian's sourcebook,None,[greenbib],978-0-8389-1504-2,1,https://www.bibsonomy.org/bibtex/2e033978aa497...,Bronkar.2017,2023-02-05 21:50:14,2023-02-16 11:47:43
39,makerspace,Publication,article,2020,"Oliver, Kevin M.","[{'first': 'Kevin M.', 'last': 'Oliver'}, {'fi...","[Kevin M. Oliver, Jennifer K. Houchins, Robert...",4,Informing Makerspace Outcomes Through a Lingui...,A growing body of research focuses on what out...,None,International Journal of Science and Mathemati...,"[assessment, outcomes, makerspace, learningana...",None,1,https://www.bibsonomy.org/bibtex/2fb48ac294725...,Oliver2020,2020-02-18 21:35:21,2020-02-18 21:35:21
40,makerspace,Publication,standard,2017,None,None,None,0,The makerspace librarian's sourcebook,The Makerspace Librarian's Sourcebook is an es...,None,None,[greenbib],978-0-8389-1504-2,1,https://www.bibsonomy.org/bibtex/2789d019603b4...,Kroski.2017,2023-02-05 21:52:55,2023-02-05 21:52:55
41,makerspace,Publication,book,2020,"Denzer, Juan","[{'first': 'Juan', 'last': 'Denzer'}, {'first'...","[Juan Denzer, Sharona Ginsberg]",2,Terrific makerspace projects: A practical guid...,Step-by-step instructions to guide you through...,None,None,[greenbib],9781538131824,1,https://www.bibsonomy.org/bibtex/2309634b7a919...,Denzer.2020,2023-02-05 16:28:34,2023-02-05 16:28:34
42,makerspace,Publication,incollection,2017,"Ginsberg, Sharona","[{'first': 'Sharona', 'last': 'Ginsberg'}]",[Sharona Ginsberg],1,Sustainability: Keeping the library makerspace...,None,The makerspace librarian's sourcebook,None,[greenbib],978-0-8389-1504-2,1,https://www.bibsonomy.org/bibtex/21c020a4a195d...,Ginsberg.2017,2023-02-16 11:58:46,2023-02-16 11:58:46


## Clean `isbn` column

In [38]:
df_clean['isbn'] = df_clean['isbn'].str.replace(r'\D+', '', regex=True)
df_clean.head()

,search_string,type,pub-type,year,author_first,authors,author,author_count,title,abstract,booktitle,journal,tags,isbn,count,id,bibtexKey,date,changeDate
38,makerspace,Publication,incollection,2017,"Bronkar, Cherie","[{'first': 'Cherie', 'last': 'Bronkar'}]",[Cherie Bronkar],1,How to start a library makerspace,You may have heard the term ``makerspace'' and...,The makerspace librarian's sourcebook,None,[greenbib],9780838915042,1,https://www.bibsonomy.org/bibtex/2e033978aa497...,Bronkar.2017,2023-02-05 21:50:14,2023-02-16 11:47:43
39,makerspace,Publication,article,2020,"Oliver, Kevin M.","[{'first': 'Kevin M.', 'last': 'Oliver'}, {'fi...","[Kevin M. Oliver, Jennifer K. Houchins, Robert...",4,Informing Makerspace Outcomes Through a Lingui...,A growing body of research focuses on what out...,None,International Journal of Science and Mathemati...,"[assessment, outcomes, makerspace, learningana...",None,1,https://www.bibsonomy.org/bibtex/2fb48ac294725...,Oliver2020,2020-02-18 21:35:21,2020-02-18 21:35:21
40,makerspace,Publication,standard,2017,None,None,None,0,The makerspace librarian's sourcebook,The Makerspace Librarian's Sourcebook is an es...,None,None,[greenbib],9780838915042,1,https://www.bibsonomy.org/bibtex/2789d019603b4...,Kroski.2017,2023-02-05 21:52:55,2023-02-05 21:52:55
41,makerspace,Publication,book,2020,"Denzer, Juan","[{'first': 'Juan', 'last': 'Denzer'}, {'first'...","[Juan Denzer, Sharona Ginsberg]",2,Terrific makerspace projects: A practical guid...,Step-by-step instructions to guide you through...,None,None,[greenbib],9781538131824,1,https://www.bibsonomy.org/bibtex/2309634b7a919...,Denzer.2020,2023-02-05 16:28:34,2023-02-05 16:28:34
42,makerspace,Publication,incollection,2017,"Ginsberg, Sharona","[{'first': 'Sharona', 'last': 'Ginsberg'}]",[Sharona Ginsberg],1,Sustainability: Keeping the library makerspace...,None,The makerspace librarian's sourcebook,None,[greenbib],9780838915042,1,https://www.bibsonomy.org/bibtex/21c020a4a195d...,Ginsberg.2017,2023-02-16 11:58:46,2023-02-16 11:58:46


## Drop duplicates

In [40]:
len(df_clean)

3149

In [42]:
df_clean = df_clean.drop_duplicates(subset='id')
len(df_clean)

3063

In [43]:
df_final = df_clean.filter(['search_string', 'pub-type', 'year', 'author_first', 'author', 'author_count',
                            'title', 'abstract', 'booktitle', 'journal', 'tags', 'isbn', 'id'])
df_final = df_final.rename(columns={'author': 'authors', 'pub-type': 'pup_type'})
df_final.head()

,search_string,pup_type,year,author_first,authors,author_count,title,abstract,booktitle,journal,tags,isbn,id
38,makerspace,incollection,2017,"Bronkar, Cherie",[Cherie Bronkar],1,How to start a library makerspace,You may have heard the term ``makerspace'' and...,The makerspace librarian's sourcebook,None,[greenbib],9780838915042,https://www.bibsonomy.org/bibtex/2e033978aa497...
39,makerspace,article,2020,"Oliver, Kevin M.","[Kevin M. Oliver, Jennifer K. Houchins, Robert...",4,Informing Makerspace Outcomes Through a Lingui...,A growing body of research focuses on what out...,None,International Journal of Science and Mathemati...,"[assessment, outcomes, makerspace, learningana...",None,https://www.bibsonomy.org/bibtex/2fb48ac294725...
40,makerspace,standard,2017,None,None,0,The makerspace librarian's sourcebook,The Makerspace Librarian's Sourcebook is an es...,None,None,[greenbib],9780838915042,https://www.bibsonomy.org/bibtex/2789d019603b4...
41,makerspace,book,2020,"Denzer, Juan","[Juan Denzer, Sharona Ginsberg]",2,Terrific makerspace projects: A practical guid...,Step-by-step instructions to guide you through...,None,None,[greenbib],9781538131824,https://www.bibsonomy.org/bibtex/2309634b7a919...
42,makerspace,incollection,2017,"Ginsberg, Sharona",[Sharona Ginsberg],1,Sustainability: Keeping the library makerspace...,None,The makerspace librarian's sourcebook,None,[greenbib],9780838915042,https://www.bibsonomy.org/bibtex/21c020a4a195d...


## Save cleaned DF to `json`

In [73]:
json_clean = Path('data/makerspace-bibsonomy-clean.json')

In [74]:
df_final.to_json(json_clean, orient='records')

In [75]:
# check, if it look correct
with open(json_clean, "r") as f:
    j_load = json.load(f)
    print(json.dumps(j_load, indent=2))

[
  {
    "search_string": "makerspace",
    "pup_type": "incollection",
    "year": 2017,
    "author_first": "Bronkar, Cherie",
    "authors": [
      "Cherie Bronkar"
    ],
    "author_count": 1,
    "title": "How to start a library makerspace",
    "abstract": "You may have heard the term ``makerspace'' and wondered what it meant. Makerspaces are, simply put, places where people gather to make things. Although that may sound like a simplistic definition, the things that can be created in a makerspace vary a great deal. Makerspaces can be high tech, low tech, and everything in between. A makerspace's offerings revolve around the needs of the community it serves, but the one thing all have in common is that they bring people together to share ideas. Typically, the first thing that comes to mind when thinking about mak- erspaces is 3D printing, but when it comes to what\u2019s going on in makerspaces around the world, that\u2019s just the tip of the iceberg. Makers create things, ide